In [35]:
import sys
stdout = sys.stdout
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdout = stdout


In [36]:
%matplotlib inline
# import sys
# reload(sys)
# sys.setdefaultencoding('utf8')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt
import scipy.stats as stats
import plotly
plotly.tools.set_credentials_file(username='prdpram08', api_key='vahwmX6mq11V2fHIeQi6')
import plotly.plotly as py
from plotly.graph_objs import *
import string
import plotly.graph_objs as go
import cufflinks as cf
import xlrd
# from sklearn.metrics import classification_report, confusion_matrix
# from sklearn.cross_validation import train_test_split
# from sklearn.linear_model import LassoLarsCV
# from sklearn.svm import SVC, LinearSVC
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.naive_bayes import GaussianNB
# from sklearn.linear_model import Perceptron
# from sklearn.linear_model import SGDClassifier
# from sklearn.tree import DecisionTreeClassifier
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.display.max_colwidth = 500
#df= pd.read_excel(open('C:\Users\pradeep.ram\Downloads\propsalanalysis\pr.xlsx','rb'), sheetname='Sheet1', encoding='ISO-8859-1')
#df= pd.read_excel(open('C:\Users\pradeep.ram\Downloads\propsalanalysis\pr.xlsx','rb'), sheetname='Sheet1', encoding='UTF-8')


In [37]:
import xlrd
file = 'data4.xlsx'
xls = xlrd.open_workbook(file)
l1=xls.sheet_names()
dfs=[]
for sheet_name in l1:
    df= pd.read_excel(open(file,'rb'), sheetname=sheet_name)
    dt=df.iloc[5,3]
    df = df.iloc[:,1:3]
    df = df.iloc[10:]
    df.columns =['Title','Description']
    df = df.reset_index(drop=True)
    for index, row in df.iterrows():
        if pd.isnull(row["Title"])==True:
            p=index
            break
    
    df = df.iloc[p+2:]    
    df.dropna(how='all',inplace=True)
    df.drop(df.tail(1).index,inplace=True)
    df.Description = df.Description.replace(np.nan, 'project', regex=True)
    df.Description=df.Description.astype(str)
    df.Title = df.Title.fillna(method='ffill')
    df["Count"]=1
    j=1
    for i in range(0,len(df)-1):
        if df.iloc[i,0]!=df.iloc[i+1,0]:
            df.iloc[i,2]=j
            j=j+1
        else:   
            df.iloc[i,2]=j
    dfg=df.groupby(['Count'])['Description'].apply(lambda x: ",".join(x)).reset_index()
    df = df.drop_duplicates(subset=['Count'])
    df.columns=["Title","Desc","Count"]
    df=pd.merge(df, dfg, on='Count')
    df.drop(['Desc','Count'], axis=1, inplace=True)
    df.iloc[0,1]='project'
    h=['Bidding Type','Client Address','Deadline','Description','Estimated Cost','Financier','Latest Update','Organisation','Project Location','Related Documents :','TI Ref. No.','Tender Notice No.']
    for index, row in df.iterrows():
        if (row["Description"]=="project") & (row["Title"] not in h):
            row["Description"]=row["Title"]
            row["Title"]="Project Title"
    df = df.set_index([df['Title'].eq('Project Title').cumsum(), 'Title'])['Description'].unstack().reset_index(drop=True)   
    df['Date']=dt
    dfs.append(df)
    df=[]

data = pd.concat(dfs,axis=0)
data = data.reset_index(drop=True)

In [38]:
data1=data[['Bidding Type',"Project Title","Description","Organisation","Project Location","Client Address",'Related Documents :',"Deadline","Estimated Cost",'Date']]
data1.head(10)

Title,Bidding Type,Project Title,Description,Organisation,Project Location,Client Address,Related Documents :,Deadline,Estimated Cost,Date
0,Domestic Competitive Bidding,"1.India - Supply, Supervision, Installation, Testing And Commissioning Of Multi Section Digital Axel Counter For 30 Detection Point And 19 Track Section Complete With All Accessories, Cable Required With Each","Tenders are invited for Supply, Supervision, Installation, Testing and commissioning of Multi section Digital Axel counter for 30 Detection point and 19 Track section complete with all accessories, Cable required with each track device and card (Viz. Axle Detectors, Track side Electronics/DAC field units, Central Evaluator (Central Evaluator unit shall be housed in a pre-fabricated Rack to be supplied along with the System) Reset unit, Relay unit, Event logger and diagnostic Terminal etc. Re...",SOUTH EASTERN RAILWAY,India,"11, Garden Reach Road, Kolkata-700043, West Bengal, Attn.:A.K.Agarwal, Controller Of Stores",Original Document : Document,2018-07-23 00:00:00,None,"Thursday, Jun 28, 2018"
1,Domestic Competitive Bidding,2.India - Supply And Commissioning Of Gps Tracking Device.,"Tenders are invited for Supply and commissioning of GPS tracking device.,Type: Open,Eligibility Criteria : -",SOUTH EASTERN RAILWAY,India,"11, Garden Reach Road, Kolkata-700043, West Bengal, Attn.:A.K.Agarwal, Controller Of Stores",Original Document : Document,2018-07-24 00:00:00,None,"Thursday, Jun 28, 2018"
2,Domestic Competitive Bidding,3.India - Providing Of Stand_alone,"Tenders are invited for Providing of Stand_alone,Tender Type: Open tender,Form Of Contract: Percentage,Tender Category: Works,No. of Covers: 2,General Technical Evaluation Allowed: No,ItemWise Technical Evaluation Allowed: No,Payment Mode: Online,Is Multi Currency Allowed For BOQ: No,Is Multi Currency Allowed For Fee: No EMD value : INR 55828 Document Purchase Start date : 29 Jun 2018 Document Purchase End date : 14 Jul 2018 Opening date : 16 Jul 2018",PUBLIC HEALTH ENGINEERING DEPARTMENT,India,"Hooghly Division, State: west Bengal Attn: Executive Engineer",Original Document,2018-07-14 00:00:00,"INR, 2791408","Thursday, Jun 28, 2018"
3,Domestic Competitive Bidding,"4.India - Emergency Restoration Of Power Supply By Replacement With New Cable Median To Street Light System, Along With Providing & Fixing Feeder Pillar Box For Diversion Of Proposed Road Adjacent Convention Ce","Tenders are invited for Emergency restoration of power supply by replacement with new cable median to street light system, along with providing & fixing Feeder Pillar Box for diversion of proposed road adjacent Convention Centre Station Left Side on MAR, New Town Kolkata under WBHIDCO.,Document cost : INR 750 Opening date : 05 Jul 2018",WEST BENGAL HOUSING INFRASTRUCTURE DEVELOPMENT CORPORATION LIMITED (HIDCO),India,"HIDCO BHABAN 35-1111 Biswa Bangla Sarani 3rd Rotary New Town Kolkata West Bengal,Tel: +91-033)-2324-6037/38,Fax: +91-(033)-2324-3016,Email_id: info@wbhidco.in",Original Document,2018-07-05 00:00:00,"INR, 192404","Thursday, Jun 28, 2018"
4,Domestic Competitive Bidding,5.India - Temporary Sitc Of Cctv And Allied It Infrastructure At Islampur Sub Divisional Court In The District Of Uttar Dinajpur During The Opening Ceremony Of The Newly Constructed Court Building Islampur Wb,Tenders are invited for Temporary SITC of CCTV and allied IT Infrastructure at Islampur Sub Divisional Court in the district of Uttar Dinajpur During the Opening ceremony of the newly Constructed Court Building Islampur WB,PUBLIC WORKS DEPARTMENT,India,Ofgfice Of the Executive Engineer North Bengal It Division PWD Complex Near 2nd Mahananda Bridge,Original Document,2018-07-04 00:00:00,None,"Thursday, Jun 28, 2018"
5,Domestic Competitive Bidding,"6.India - Maintenance Of Led Street Lights Of Action Area-ii, Excluding Mar, At Newtown Under Nkda 01/07/02018 To 30/06/2019 I.e. 365 Days.","Tenders are invited for Maintenance of led street lights of action are

In [39]:
data1["DescriptionA"]=data1["Description"]

In [40]:
data1['DescriptionA'] = data1['DescriptionA'].str.replace(',', ' ')
# data1['DescriptionA'] = data1['DescriptionA'].str.replace('.', ' ')
data1['DescriptionA'] = data1['DescriptionA'].str.replace('-', ' ')
data1['DescriptionA'] = data1['DescriptionA'].str.replace('(', ' ')
data1['DescriptionA'] = data1['DescriptionA'].str.replace(')', ' ')
data1['DescriptionA'] = data1['DescriptionA'].str.replace(':', ' ')
data1['DescriptionA'] = data1['DescriptionA'].str.replace(';', ' ')
data1['DescriptionA'] = data1['DescriptionA'].str.replace('"', ' ')
data1['DescriptionA'] = data1['DescriptionA'].str.replace('/', ' ')
data1['DescriptionA'] = data1['DescriptionA'].str.replace('%', ' ')
data1['DescriptionA'] = data1['DescriptionA'].str.replace('&', ' ')
data1['DescriptionA'] = data1['DescriptionA'].str.replace("'", ' ')
data1['DescriptionA'] = data1['DescriptionA'].str.replace(".", ' ')
data1['DescriptionA'] = data1['DescriptionA'].str.lower()
data1.head()

Title,Bidding Type,Project Title,Description,Organisation,Project Location,Client Address,Related Documents :,Deadline,Estimated Cost,Date,DescriptionA
0,Domestic Competitive Bidding,"1.India - Supply, Supervision, Installation, Testing And Commissioning Of Multi Section Digital Axel Counter For 30 Detection Point And 19 Track Section Complete With All Accessories, Cable Required With Each","Tenders are invited for Supply, Supervision, Installation, Testing and commissioning of Multi section Digital Axel counter for 30 Detection point and 19 Track section complete with all accessories, Cable required with each track device and card (Viz. Axle Detectors, Track side Electronics/DAC field units, Central Evaluator (Central Evaluator unit shall be housed in a pre-fabricated Rack to be supplied along with the System) Reset unit, Relay unit, Event logger and diagnostic Terminal etc. Re...",SOUTH EASTERN RAILWAY,India,"11, Garden Reach Road, Kolkata-700043, West Bengal, Attn.:A.K.Agarwal, Controller Of Stores",Original Document : Document,2018-07-23 00:00:00,None,"Thursday, Jun 28, 2018",tenders are invited for supply supervision installation testing and commissioning of multi section digital axel counter for 30 detection point and 19 track section complete with all accessories cable required with each track device and card viz axle detectors track side electronics dac field units central evaluator central evaluator unit shall be housed in a pre fabricated rack to be supplied along with the system reset unit relay unit event logger and diagnostic terminal etc re...
1,Domestic Competitive Bidding,2.India - Supply And Commissioning Of Gps Tracking Device.,"Tenders are invited for Supply and commissioning of GPS tracking device.,Type: Open,Eligibility Criteria : -",SOUTH EASTERN RAILWAY,India,"11, Garden Reach Road, Kolkata-700043, West Bengal, Attn.:A.K.Agarwal, Controller Of Stores",Original Document : Document,2018-07-24 00:00:00,None,"Thursday, Jun 28, 2018",tenders are invited for supply and commissioning of gps tracking device type open eligibility criteria
2,Domestic Competitive Bidding,3.India - Providing Of Stand_alone,"Tenders are invited for Providing of Stand_alone,Tender Type: Open tender,Form Of Contract: Percentage,Tender Category: Works,No. of Covers: 2,General Technical Evaluation Allowed: No,ItemWise Technical Evaluation Allowed: No,Payment Mode: Online,Is Multi Currency Allowed For BOQ: No,Is Multi Currency Allowed For Fee: No EMD value : INR 55828 Document Purchase Start date : 29 Jun 2018 Document Purchase End date : 14 Jul 2018 Opening date : 16 Jul 2018",PUBLIC HEALTH ENGINEERING DEPARTMENT,India,"Hooghly Division, State: west Bengal Attn: Executive Engineer",Original Document,2018-07-14 00:00:00,"INR, 2791408","Thursday, Jun 28, 2018",tenders are invited for providing of stand_alone tender type open tender form of contract percentage tender category works no of covers 2 general technical evaluation allowed no itemwise technical evaluation allowed no payment mode online is multi currency allowed for boq no is multi currency allowed for fee no emd value inr 55828 document purchase start date 29 jun 2018 document purchase end date 14 jul 2018 opening date 16 jul 2018
3,Domestic Competitive Bidding,"4.India - Emergency Restoration Of Power Supply By Replacement With New Cable Median To Street Light System, Along With Providing & Fixing Feeder Pillar Box For Diversion Of Proposed Road Adjacent Convention Ce","Tenders are invited for Emergency restoration of power supply by replacement with new cable median to street light system, along with providing & fixing Feeder Pillar Box for diversion of proposed road adjacent Convention Centre Station Left Side on MAR, New Town Kolkata under WBHIDCO.,Document cost : INR 750 Opening date : 05 Jul 2018",WEST BENGAL HOUSING INFRASTRUCTURE DEVELOPMENT CORPORATION LIMITED (HIDCO),India,"HIDCO BHABAN 35-1111 Biswa Bangla Sarani 3rd Rotary New Town Kolkata West Bengal,Tel: +91-033)-232

In [41]:
data1.dtypes

Title
Bidding Type           object
Project Title          object
Description            object
Organisation           object
Project Location       object
Client Address         object
Related Documents :    object
Deadline               object
Estimated Cost         object
Date                   object
DescriptionA           object
dtype: object

In [42]:
import nltk
# nltk.download('wordnet')
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

# def lemmatize_text(text):
#     return [lemmatizer.lemmatize(w,'v') for w in w_tokenizer.tokenize(text)]

from nltk.stem import PorterStemmer
porter = PorterStemmer()
def stem_text(text):
    return [porter.stem(w) for w in w_tokenizer.tokenize(text.decode('utf-8'))]
# i1['text_lemmatized'] = i1.Title.apply(lemmatize_text)
#df['text_lemmatized2'] = df.Title.apply(lemmatize_text)
data1['text_stem'] = data1["DescriptionA"].apply(stem_text)
data1.head()

Title,Bidding Type,Project Title,Description,Organisation,Project Location,Client Address,Related Documents :,Deadline,Estimated Cost,Date,DescriptionA,text_stem
0,Domestic Competitive Bidding,"1.India - Supply, Supervision, Installation, Testing And Commissioning Of Multi Section Digital Axel Counter For 30 Detection Point And 19 Track Section Complete With All Accessories, Cable Required With Each","Tenders are invited for Supply, Supervision, Installation, Testing and commissioning of Multi section Digital Axel counter for 30 Detection point and 19 Track section complete with all accessories, Cable required with each track device and card (Viz. Axle Detectors, Track side Electronics/DAC field units, Central Evaluator (Central Evaluator unit shall be housed in a pre-fabricated Rack to be supplied along with the System) Reset unit, Relay unit, Event logger and diagnostic Terminal etc. Re...",SOUTH EASTERN RAILWAY,India,"11, Garden Reach Road, Kolkata-700043, West Bengal, Attn.:A.K.Agarwal, Controller Of Stores",Original Document : Document,2018-07-23 00:00:00,None,"Thursday, Jun 28, 2018",tenders are invited for supply supervision installation testing and commissioning of multi section digital axel counter for 30 detection point and 19 track section complete with all accessories cable required with each track device and card viz axle detectors track side electronics dac field units central evaluator central evaluator unit shall be housed in a pre fabricated rack to be supplied along with the system reset unit relay unit event logger and diagnostic terminal etc re...,"[tender, are, invit, for, suppli, supervis, instal, test, and, commiss, of, multi, section, digit, axel, counter, for, 30, detect, point, and, 19, track, section, complet, with, all, accessori, cabl, requir, with, each, track, devic, and, card, viz, axl, detector, track, side, electron, dac, field, unit, central, evalu, central, evalu, unit, shall, be, hous, in, a, pre, fabric, rack, to, be, suppli, along, with, the, system, reset, unit, relay, unit, event, logger, and, diagnost, termin, etc..."
1,Domestic Competitive Bidding,2.India - Supply And Commissioning Of Gps Tracking Device.,"Tenders are invited for Supply and commissioning of GPS tracking device.,Type: Open,Eligibility Criteria : -",SOUTH EASTERN RAILWAY,India,"11, Garden Reach Road, Kolkata-700043, West Bengal, Attn.:A.K.Agarwal, Controller Of Stores",Original Document : Document,2018-07-24 00:00:00,None,"Thursday, Jun 28, 2018",tenders are invited for supply and commissioning of gps tracking device type open eligibility criteria,"[tender, are, invit, for, suppli, and, commiss, of, gp, track, devic, type, open, elig, criteria]"
2,Domestic Competitive Bidding,3.India - Providing Of Stand_alone,"Tenders are invited for Providing of Stand_alone,Tender Type: Open tender,Form Of Contract: Percentage,Tender Category: Works,No. of Covers: 2,General Technical Evaluation Allowed: No,ItemWise Technical Evaluation Allowed: No,Payment Mode: Online,Is Multi Currency Allowed For BOQ: No,Is Multi Currency Allowed For Fee: No EMD value : INR 55828 Document Purchase Start date : 29 Jun 2018 Document Purchase End date : 14 Jul 2018 Opening date : 16 Jul 2018",PUBLIC HEALTH ENGINEERING DEPARTMENT,India,"Hooghly Division, State: west Bengal Attn: Executive Engineer",Original Document,2018-07-14 00:00:00,"INR, 2791408","Thursday, Jun 28, 2018",tenders are invited for providing of stand_alone tender type open tender form of contract percentage tender category works no of covers 2 general technical evaluation allowed no itemwise technical evaluation allowed no payment mode online is multi currency allowed for boq no is multi currency allowed for fee no emd value inr 55828 document purchase start date 29 jun 2018 document purchase end date 14 jul 2018 opening date 16 jul 2018,"[tender, are, invit, for, provid, of, stand_alon, tender, type, open, tender, form, of, contract, percentag, tender, categori, work, no, of, cover, 2, 

In [43]:
# import nltk
# # nltk.download('stopwords')
# from nltk.corpus import stopwords
# stop = stopwords.words('english')
# data1['text_stem']=data1['text_stem'].apply(lambda x: [item for item in x if item not in stop])
# data1.head()

In [44]:
# data1['text']=data1.text_new.apply(lambda x: ' '.join(x))

In [45]:
data1["BU"]=" "

In [46]:
enggword=['detail','prepar','road']
enggword1=['dpr','flyover']
enggword2=['dpr','by-pass']
enggword3=['locat','survey','digit','model']
enggword4=['project','manag','consult','hospit']
enggword5=['consult','road']
enggword6=['detail','report','bridg']
enggword7=['dpr','bridg']
enggword8=['dpr','road']
enggword9=['detail','report','flyover']
enggword10=['consult','structur']
enggword11=['consult','build']
enggword12=['consult','hous']
enggword13=['detail','report','survey']
enggword14=['develop', 'multimod', 'transit','hub']
enggword15=['detail','report','bu']
enggword16=['detail','prepar','lane']
enggword17=['consult','station']
enggword18=['consult','lane']
enggword19=['dpr','build']
enggword20=['prepar','draw','survey']
enggword21=['consult','architectur']
enggword22=['consult', 'prepar', 'zonal', 'develop', 'plan']
enggword23=['consult','landscap']
enggword24=['consult','packag']
enggword25=['consult','highway']
enggword26=['consult','bridg']
enggword27=["feasibl","studi","road"]
enggword28=['consult','terminu']
enggword29=['feasibl', 'studi','lane']
enggword30=['design', 'draw','road']
enggword31=['design', 'draw','lane']
enggword32=["studi","lane"]
enggword33=["consult","km"]
enggword34=["preliminari","km"]
enggword35=['consult','stand','bu']
enggword36=['consult','complex']
enggword37=['consult','develop','infrastructur']
enggword38=['consult','pmay']
enggword39=['consult','expressway']
enggword40=["studi","road"]
enggword41=["detail","report","complex"]
enggword42=["geolog","investig"]
enggword43=["consult","route"]
enggword44=["dpr","bu","stand"]
enggword45=["geotechn","studi"]
enggword46=["consult","bypass"]
enggword47=["geotechn","investig"]
enggword48=["topograph","survey"]
enggword49=["studi","build"]
railword=['detail','report','rail']
railword2=['detail','report','railway']
railword3=['consult','rail']
railword4=['consult','railway']
railword5=["studi","railway"]
railword6=["preliminari","railway"]
railword7=["studi","rail"]
railword8=["pmc","rail"]
railword9=["pmc","railway"]
railword10=['consult','side']
railword11=["design","draw","bridg"]
railword12=["estim","bridg"]
railword13=["consult","stn"]
railword14=["gad",'railway']
railword15=["plan",'railway']
bdword4=['consult','smart','citi']
# ttsword1=['detail','consult','smart','citi']
itsword1=['consult', 'automat', 'traffic','control']
itsword2=['master','system', 'integr']
itsword3=['integr','traffic']
itsword4=['traffic', 'signal']
itsword5=['instal', 'traffic']
itsword6=['integr','transport','system']
itsword7=['intellig','transport']
itsword8=['si','smart','citi']
itsword9=['traffic', 'control']
itsword10=['traffic', 'manag','system']
itsword=['repair','traffic','signal']
itword1=['ticket','bu','platform']
itword2=['implement','control','centr']
itword3=["vehicl","track"]
ttsword4=['consult','command','centr']
# ttsword6=['consult','smart','citi']
ttsword6=['consult','transport','implement']
# itword5=['system','integr']
ttsword5=['system','integr','consult','traffic']
itword6=['ict','master','plan']
ttsword7=['consult','cctv']
ttsword8=['consult','it']
itword8=['procur','gps']
itword9=['track','gps']
itword10=['intellig','transit']
itword11=['fare','system','collect']
erpword1=['erp','consult']
tpword1=['consult','transport']
tpword2=['consult','traffic']
tpword3=['consult','freight']
tpword4=['safeti','consult']
tpword5=['consult','multimod']
tpword6=["consult", "master","plan"]
tpword7=["consult", "mobil","plan"]
tpword8=["consult", "pedestrian"]
tpword9=["transport", "plan"]
tpword10=["urban", "plan"]
tpword11=["town", "plan"]
tpword12=["provis","transport","servic"]
tpword13=["region","plan"]
tpword14=["develop","road","plan"]
tpword15=["traffic","plan"]
tpword16=["develop","plan","prepar"]
tpword17=["integr","plan","develop"]
tpword18=["dpr","pedestrian"]
tpword19=["transport","studi"]
tpword20=["transport","document"]
tpword21=["traffic","impact"]
tpword22=['detail','report','freight']
tpword23=['master','plan']
tpword24=['road','safeti','audit']
tpword25=['socio','econom','impact']
tpword26=['transit','servic']
tpword27=['detail','report','town']
tpword28=['land','plan']
bpword1=["transact", "advisori"]
bpword2=["consult","car" "park"]
bpword3=["transact", "advisor"]
bpword4=["studi","car" "park"]
bpword5=["preliminari","car" "park"]
bpword6=["consult","ppp"]
bpword7=["fund","expert"]
bdword1=["transport","traffic","signal","transit",'route','bu','ticket']
bdword2=["monitor","project","consult"]
bdword3=["project","manag","consult"]

opword1=["implement", "park"]
opword2=["instal", "park"]
opword3=["manag", "bu","stand"]
opword4=["drive", "licens"]
rtword1=["consult", "bu"]
rtword2=['electr', 'buse' ]
rtword3=['electr', 'bu'] 
rtword4=['electr','vehicl']
stopword=['author','engin']
stopword2=['independ','engin']
stopword3=['authority', 'engin']
# stopword4=['supervis', 'consult', 'servic','lane']
stopword5=['consult','ae']
stopword6=['consult','ie']
stopword7=['dpr','lidar']
stopword8=['water','drainag']
stopword9=['system','integr','water']
stopword10=['water','expert']
stopword11=['water','suppli']
stopword12=['system','integr','child']
# stopword13=['system','integr','document']
stopword14=['satellit','track']
stopword15=['system','integr','secur']
stopword16=['shop','plan']
stopword17=['librari','academ','disabl','child','watershed','coal','drug','oil','fire','azur','insur','water','univers','lake','canal','geodetic']

for index, row in data1.iterrows():
    m=row["text_stem"]
    if all(word in m  for word in opword1):
        row["BU"]="OP" 
    if all(word in m  for word in opword2):
        row["BU"]="OP"
    if all(word in m  for word in opword3):
        row["BU"]="OP" 
    if all(word in m  for word in opword4):
        row["BU"]="OP" 
    if any(word in m  for word in bdword1):
        row["BU"]="BD2" 
    if all(word in m  for word in bdword2):
        row["BU"]="BD2" 
    if all(word in m  for word in bdword3):
        row["BU"]="BD2" 
    if all(word in m  for word in bdword4):
        row["BU"]="BD"  
    if all(word in m  for word in tpword3):
        row["BU"]="TP" 
    if all(word in m  for word in tpword5):
        row["BU"]="TP"
    if all(word in m  for word in tpword6):
        row["BU"]="TP" 
    if all(word in m  for word in tpword7):
        row["BU"]="TP"
    if all(word in m  for word in tpword8):
        row["BU"]="TP"
    if all(word in m  for word in tpword9):
        row["BU"]="TP" 
    if all(word in m  for word in tpword10):
        row["BU"]="TP"
    if all(word in m  for word in tpword11):
        row["BU"]="TP" 
    if all(word in m  for word in tpword12):
        row["BU"]="TP" 
    if all(word in m  for word in tpword13):
        row["BU"]="TP" 
    if all(word in m  for word in tpword14):
        row["BU"]="TP"
    if all(word in m  for word in tpword15):
        row["BU"]="TP" 
    if all(word in m  for word in tpword16):
        row["BU"]="TP"   
    if all(word in m  for word in tpword18):
        row["BU"]="TP" 
    if all(word in m  for word in tpword19):
        row["BU"]="TP" 
    if all(word in m  for word in tpword20):
        row["BU"]="TP"
    if all(word in m  for word in tpword21):
        row["BU"]="TP"  
    if all(word in m  for word in tpword22):
        row["BU"]="TP" 
    if all(word in m  for word in tpword23):
        row["BU"]="TP" 
    if all(word in m  for word in tpword24):
        row["BU"]="TP"  
    if all(word in m  for word in tpword25):
        row["BU"]="TP" 
    if all(word in m  for word in tpword26):
        row["BU"]="TP" 
    if all(word in m  for word in tpword27):
        row["BU"]="TP"
    if all(word in m  for word in tpword28):
        row["BU"]="TP"     
    if all(word in m  for word in rtword1):
        row["BU"]="RT" 
    if all(word in m  for word in rtword2):
        row["BU"]="RT" 
    if all(word in m  for word in rtword3):
        row["BU"]="RT" 
    if all(word in m  for word in rtword4):
        row["BU"]="RT"   
    if all(word in m  for word in ttsword5 ):
        row["BU"]="TTS" 

    if all(word in m  for word in ttsword7 ):
        row["BU"]="TTS"         
#     if all(word in m  for word in ttsword8 ):
#         row["BU"]="TTS"      
    if all(word in m  for word in enggword ):
        row["BU"]="Engg"
    if all(word in m  for word in enggword1 ):
        row["BU"]="Engg"  
    if all(word in m  for word in enggword2 ):
        row["BU"]="Engg" 
    if all(word in m  for word in enggword3 ):
        row["BU"]="Engg"     
    if all(word in m  for word in enggword4 ):
        row["BU"]="Engg" 
    if all(word in m  for word in enggword5 ):
        row["BU"]="Engg"  
    if all(word in m  for word in enggword6 ):
        row["BU"]="Engg"
    if all(word in m  for word in enggword7 ):
        row["BU"]="Engg"
    if all(word in m  for word in enggword8 ):
        row["BU"]="Engg"
    if all(word in m  for word in enggword9 ):
        row["BU"]="Engg"  
    if all(word in m  for word in enggword10 ):
        row["BU"]="Engg" 
    if all(word in m  for word in enggword11 ):
        row["BU"]="Engg" 
    if all(word in m  for word in enggword12 ):
        row["BU"]="Engg" 
    if all(word in m  for word in enggword13):
        row["BU"]="Engg"     
    if all(word in m  for word in enggword14):
        row["BU"]="Engg"
    if all(word in m  for word in enggword15):
        row["BU"]="Engg"
    if all(word in m  for word in enggword16):
        row["BU"]="Engg"   
    if all(word in m  for word in enggword17):
        row["BU"]="Engg" 
    if all(word in m  for word in enggword18):
        row["BU"]="Engg"  
    if all(word in m  for word in enggword19):
        row["BU"]="Engg"  
    if all(word in m  for word in enggword20):
        row["BU"]="Engg" 
    if all(word in m  for word in enggword21):
        row["BU"]="Engg" 
    if all(word in m  for word in enggword22):
        row["BU"]="Engg"
    if all(word in m  for word in enggword23):
        row["BU"]="Engg"
#     if all(word in m  for word in enggword24):
#         row["BU"]="Engg"
    if all(word in m  for word in enggword25):
        row["BU"]="Engg"
    if all(word in m  for word in enggword26):
        row["BU"]="Engg"
    if all(word in m  for word in enggword27):
        row["BU"]="Engg"  
    if all(word in m  for word in enggword28):
        row["BU"]="Engg" 
    if all(word in m  for word in enggword29):
        row["BU"]="Engg"   
    if all(word in m  for word in enggword30):
        row["BU"]="Engg" 
    if all(word in m  for word in enggword31):
        row["BU"]="Engg" 
    if all(word in m  for word in enggword32):
        row["BU"]="Engg"
    if all(word in m  for word in enggword33):
        row["BU"]="Engg"
    if all(word in m  for word in enggword34):
        row["BU"]="Engg"
    if all(word in m  for word in enggword35):
        row["BU"]="Engg"  
    if all(word in m  for word in enggword36):
        row["BU"]="Engg"
    if all(word in m  for word in enggword37):
        row["BU"]="Engg"
    if all(word in m  for word in enggword38):
        row["BU"]="Engg"
    if all(word in m  for word in enggword39):
        row["BU"]="Engg" 
    if all(word in m  for word in enggword40):
        row["BU"]="Engg"
    if all(word in m  for word in enggword41):
        row["BU"]="Engg"  
    if all(word in m  for word in enggword42):
        row["BU"]="Engg" 
    if all(word in m  for word in enggword43):
        row["BU"]="Engg" 
    if all(word in m  for word in enggword44):
        row["BU"]="Engg"  
    if all(word in m  for word in enggword45):
        row["BU"]="Engg"
    if all(word in m  for word in enggword46):
        row["BU"]="Engg"
    if all(word in m  for word in enggword47):
        row["BU"]="Engg"
    if all(word in m  for word in enggword48):
        row["BU"]="Engg" 
    if all(word in m  for word in enggword49):
        row["BU"]="Engg"     
    if all(word in m  for word in tpword2):
        row["BU"]="TP"  
    if all(word in m  for word in tpword19):
        row["BU"]="TP"     
    if all(word in m  for word in railword ):
        row["BU"]="Rail" 
    if all(word in m  for word in railword2 ):
        row["BU"]="Rail"  
    if all(word in m  for word in railword3 ):
        row["BU"]="Rail"    
    if all(word in m  for word in railword4 ):
        row["BU"]="Rail" 
    if all(word in m  for word in railword5 ):
        row["BU"]="Rail"  
    if all(word in m  for word in railword6 ):
        row["BU"]="Rail"  
    if all(word in m  for word in railword7 ):
        row["BU"]="Rail" 
    if all(word in m  for word in railword8 ):
        row["BU"]="Rail"
    if all(word in m  for word in railword9 ):
        row["BU"]="Rail" 
    if all(word in m  for word in railword10 ):
        row["BU"]="Rail"  
    if all(word in m  for word in railword11 ):
        row["BU"]="Rail" 
    if all(word in m  for word in railword12 ):
        row["BU"]="Rail"  
    if all(word in m  for word in railword13 ):
        row["BU"]="Rail"
    if all(word in m  for word in railword14):
        row["BU"]="Rail" 
    if all(word in m  for word in railword15):
        row["BU"]="Rail"     
    if all(word in m  for word in tpword4):
        row["BU"]="TP"     
#     if all(word in m  for word in ttsword1 ):
#         row["BU"]="TTS" 

#     if all(word in m  for word in ttsword2 ):
#         row["BU"]="TTS"     
    if all(word in m  for word in itsword1 ):
        row["BU"]="ITS"
    if all(word in m  for word in itsword2 ):
        row["BU"]="ITS" 
    if all(word in m  for word in itsword3 ):
        row["BU"]="ITS"
    if all(word in m  for word in itsword4 ):
        row["BU"]="ITS" 
    if all(word in m  for word in itsword5 ):
        row["BU"]="ITS" 
    if all(word in m  for word in itsword6 ):
        row["BU"]="ITS"
    if all(word in m  for word in itsword8 ):
        row["BU"]="ITS"
    if all(word in m  for word in itsword9 ):
        row["BU"]="ITS" 
    if all(word in m  for word in itsword10 ):
        row["BU"]="ITS"    
    if all(word in m  for word in itword10 ):
        row["BU"]="IT"   
    if all(word in m  for word in itword11 ):
        row["BU"]="IT"     
    if all(word in m  for word in itword1):
        row["BU"]="IT" 
    if all(word in m  for word in itword2):
        row["BU"]="IT"
    if all(word in m  for word in itword3):
        row["BU"]="IT" 
    if all(word in m  for word in ttsword4):
        row["BU"]="TTS"   
#     if all(word in m  for word in itword5):
#         row["BU"]="IT"
    if all(word in m  for word in itword6):
        row["BU"]="IT"   
    if all(word in m  for word in ttsword7):
        row["BU"]="TTS"     
    if all(word in m  for word in itsword7 ):
        row["BU"]="ITS"     
    if all(word in m  for word in tpword1):
        row["BU"]="TP"    
    if all(word in m  for word in erpword1):
        row["BU"]="TTS" 
    if all(word in m  for word in bpword2):
        row["BU"]="BP" 
    if all(word in m  for word in bpword4):
        row["BU"]="BP"
    if all(word in m  for word in bpword5):
        row["BU"]="BP" 
    if all(word in m  for word in bpword6):
        row["BU"]="BP"  
    if all(word in m  for word in bpword7):
        row["BU"]="BP"    

    if all(word in m  for word in ttsword6 ):
        row["BU"]="TTS"    
#     if all(word in m  for word in stopword):
#         row["BU"]=" " 
#     if all(word in m  for word in stopword2):
#         row["BU"]=" " 
#     if all(word in m  for word in stopword3):
#         row["BU"]=" "  
#     if all(word in m  for word in stopword4):
#         row["BU"]=" " 
#     if all(word in m  for word in stopword5):
#         row["BU"]=" " 
#     if all(word in m  for word in stopword6):
#         row["BU"]=" " 
#     if all(word in m  for word in stopword7):
#         row["BU"]=" " 
    if all(word in m  for word in stopword8):
        row["BU"]=" " 
    if all(word in m  for word in stopword9):
        row["BU"]=" "
    if all(word in m  for word in stopword10):
        row["BU"]=" "
    if all(word in m  for word in stopword11):
        row["BU"]=" " 
#     if all(word in m  for word in stopword12):
#         row["BU"]=" "
#     if all(word in m  for word in stopword13):
#         row["BU"]=" " 
#     if all(word in m  for word in stopword14):
#         row["BU"]=" " 
#     if all(word in m  for word in stopword15):
#         row["BU"]=" "  
#     if all(word in m  for word in stopword16):
#         row["BU"]=" "
    if any(word in m  for word in stopword17):
        row["BU"]=" " 
    if all(word in m  for word in bpword3):
        row["BU"]="BP" 
    if all(word in m  for word in bpword1):
        row["BU"]="BP"  
    if all(word in m  for word in tpword9):
        row["BU"]="TP"     
# data1[["Project Title","Project Location","BU","Description","Estimated Cost","Organisation","text_stem"]]

In [47]:
data1.head(50)

Title,Bidding Type,Project Title,Description,Organisation,Project Location,Client Address,Related Documents :,Deadline,Estimated Cost,Date,DescriptionA,text_stem,BU
0,Domestic Competitive Bidding,"1.India - Supply, Supervision, Installation, Testing And Commissioning Of Multi Section Digital Axel Counter For 30 Detection Point And 19 Track Section Complete With All Accessories, Cable Required With Each","Tenders are invited for Supply, Supervision, Installation, Testing and commissioning of Multi section Digital Axel counter for 30 Detection point and 19 Track section complete with all accessories, Cable required with each track device and card (Viz. Axle Detectors, Track side Electronics/DAC field units, Central Evaluator (Central Evaluator unit shall be housed in a pre-fabricated Rack to be supplied along with the System) Reset unit, Relay unit, Event logger and diagnostic Terminal etc. Re...",SOUTH EASTERN RAILWAY,India,"11, Garden Reach Road, Kolkata-700043, West Bengal, Attn.:A.K.Agarwal, Controller Of Stores",Original Document : Document,2018-07-23 00:00:00,None,"Thursday, Jun 28, 2018",tenders are invited for supply supervision installation testing and commissioning of multi section digital axel counter for 30 detection point and 19 track section complete with all accessories cable required with each track device and card viz axle detectors track side electronics dac field units central evaluator central evaluator unit shall be housed in a pre fabricated rack to be supplied along with the system reset unit relay unit event logger and diagnostic terminal etc re...,"[tender, are, invit, for, suppli, supervis, instal, test, and, commiss, of, multi, section, digit, axel, counter, for, 30, detect, point, and, 19, track, section, complet, with, all, accessori, cabl, requir, with, each, track, devic, and, card, viz, axl, detector, track, side, electron, dac, field, unit, central, evalu, central, evalu, unit, shall, be, hous, in, a, pre, fabric, rack, to, be, suppli, along, with, the, system, reset, unit, relay, unit, event, logger, and, diagnost, termin, etc...",
1,Domestic Competitive Bidding,2.India - Supply And Commissioning Of Gps Tracking Device.,"Tenders are invited for Supply and commissioning of GPS tracking device.,Type: Open,Eligibility Criteria : -",SOUTH EASTERN RAILWAY,India,"11, Garden Reach Road, Kolkata-700043, West Bengal, Attn.:A.K.Agarwal, Controller Of Stores",Original Document : Document,2018-07-24 00:00:00,None,"Thursday, Jun 28, 2018",tenders are invited for supply and commissioning of gps tracking device type open eligibility criteria,"[tender, are, invit, for, suppli, and, commiss, of, gp, track, devic, type, open, elig, criteria]",
2,Domestic Competitive Bidding,3.India - Providing Of Stand_alone,"Tenders are invited for Providing of Stand_alone,Tender Type: Open tender,Form Of Contract: Percentage,Tender Category: Works,No. of Covers: 2,General Technical Evaluation Allowed: No,ItemWise Technical Evaluation Allowed: No,Payment Mode: Online,Is Multi Currency Allowed For BOQ: No,Is Multi Currency Allowed For Fee: No EMD value : INR 55828 Document Purchase Start date : 29 Jun 2018 Document Purchase End date : 14 Jul 2018 Opening date : 16 Jul 2018",PUBLIC HEALTH ENGINEERING DEPARTMENT,India,"Hooghly Division, State: west Bengal Attn: Executive Engineer",Original Document,2018-07-14 00:00:00,"INR, 2791408","Thursday, Jun 28, 2018",tenders are invited for providing of stand_alone tender type open tender form of contract percentage tender category works no of covers 2 general technical evaluation allowed no itemwise technical evaluation allowed no payment mode online is multi currency allowed for boq no is multi currency allowed for fee no emd value inr 55828 document purchase start date 29 jun 2018 document purchase end date 14 jul 2018 opening date 16 jul 2018,"[tender, are, invit, for, provid, of, stand_alon, tender, type, open, tender, form, of, contract, percentag, tender, categori, work, no, of, cover

In [48]:
# from nltk import PorterStemmer
PorterStemmer().stem('university')

u'univers'

In [49]:
data1["BD"]=" "
data1.head()

Title,Bidding Type,Project Title,Description,Organisation,Project Location,Client Address,Related Documents :,Deadline,Estimated Cost,Date,DescriptionA,text_stem,BU,BD
0,Domestic Competitive Bidding,"1.India - Supply, Supervision, Installation, Testing And Commissioning Of Multi Section Digital Axel Counter For 30 Detection Point And 19 Track Section Complete With All Accessories, Cable Required With Each","Tenders are invited for Supply, Supervision, Installation, Testing and commissioning of Multi section Digital Axel counter for 30 Detection point and 19 Track section complete with all accessories, Cable required with each track device and card (Viz. Axle Detectors, Track side Electronics/DAC field units, Central Evaluator (Central Evaluator unit shall be housed in a pre-fabricated Rack to be supplied along with the System) Reset unit, Relay unit, Event logger and diagnostic Terminal etc. Re...",SOUTH EASTERN RAILWAY,India,"11, Garden Reach Road, Kolkata-700043, West Bengal, Attn.:A.K.Agarwal, Controller Of Stores",Original Document : Document,2018-07-23 00:00:00,None,"Thursday, Jun 28, 2018",tenders are invited for supply supervision installation testing and commissioning of multi section digital axel counter for 30 detection point and 19 track section complete with all accessories cable required with each track device and card viz axle detectors track side electronics dac field units central evaluator central evaluator unit shall be housed in a pre fabricated rack to be supplied along with the system reset unit relay unit event logger and diagnostic terminal etc re...,"[tender, are, invit, for, suppli, supervis, instal, test, and, commiss, of, multi, section, digit, axel, counter, for, 30, detect, point, and, 19, track, section, complet, with, all, accessori, cabl, requir, with, each, track, devic, and, card, viz, axl, detector, track, side, electron, dac, field, unit, central, evalu, central, evalu, unit, shall, be, hous, in, a, pre, fabric, rack, to, be, suppli, along, with, the, system, reset, unit, relay, unit, event, logger, and, diagnost, termin, etc...",,
1,Domestic Competitive Bidding,2.India - Supply And Commissioning Of Gps Tracking Device.,"Tenders are invited for Supply and commissioning of GPS tracking device.,Type: Open,Eligibility Criteria : -",SOUTH EASTERN RAILWAY,India,"11, Garden Reach Road, Kolkata-700043, West Bengal, Attn.:A.K.Agarwal, Controller Of Stores",Original Document : Document,2018-07-24 00:00:00,None,"Thursday, Jun 28, 2018",tenders are invited for supply and commissioning of gps tracking device type open eligibility criteria,"[tender, are, invit, for, suppli, and, commiss, of, gp, track, devic, type, open, elig, criteria]",,
2,Domestic Competitive Bidding,3.India - Providing Of Stand_alone,"Tenders are invited for Providing of Stand_alone,Tender Type: Open tender,Form Of Contract: Percentage,Tender Category: Works,No. of Covers: 2,General Technical Evaluation Allowed: No,ItemWise Technical Evaluation Allowed: No,Payment Mode: Online,Is Multi Currency Allowed For BOQ: No,Is Multi Currency Allowed For Fee: No EMD value : INR 55828 Document Purchase Start date : 29 Jun 2018 Document Purchase End date : 14 Jul 2018 Opening date : 16 Jul 2018",PUBLIC HEALTH ENGINEERING DEPARTMENT,India,"Hooghly Division, State: west Bengal Attn: Executive Engineer",Original Document,2018-07-14 00:00:00,"INR, 2791408","Thursday, Jun 28, 2018",tenders are invited for providing of stand_alone tender type open tender form of contract percentage tender category works no of covers 2 general technical evaluation allowed no itemwise technical evaluation allowed no payment mode online is multi currency allowed for boq no is multi currency allowed for fee no emd value inr 55828 document purchase start date 29 jun 2018 document purchase end date 14 jul 2018 opening date 16 jul 2018,"[tender, are, invit, for, provid, of, stand_alon, tender, type, open, tender, form, of, contract, percentag, tender, categori, work, no, of, 

In [50]:
bu1=['TP','OP']
bu2=['Rail','Engg','BP']
bu3=['IT','TTS']
bu4=['IT','ITS']
bu5=['RT']
bu6=['BD']
bu7=['BD2']
for index, row in data1.iterrows():
    m=row["BU"]
    if m in bu1:
        row['BD']='Ashish'   
    if m in bu2:
        row['BD']='Anand'      
    if m in bu3:
        row['BD']='Saumya' 
    if m in bu4:
        row['BD']='Gurpreet'  
    if m in bu5:
        row['BD']='Sudeep'  
    if m in bu6:
        row['BD']='All'
    if m in bu7:
        row['BD']='Check'     

In [51]:
data1['Estimated Cost'] = data1['Estimated Cost'].str.replace("INR,", 'Rs.')
data1.head()

Title,Bidding Type,Project Title,Description,Organisation,Project Location,Client Address,Related Documents :,Deadline,Estimated Cost,Date,DescriptionA,text_stem,BU,BD
0,Domestic Competitive Bidding,"1.India - Supply, Supervision, Installation, Testing And Commissioning Of Multi Section Digital Axel Counter For 30 Detection Point And 19 Track Section Complete With All Accessories, Cable Required With Each","Tenders are invited for Supply, Supervision, Installation, Testing and commissioning of Multi section Digital Axel counter for 30 Detection point and 19 Track section complete with all accessories, Cable required with each track device and card (Viz. Axle Detectors, Track side Electronics/DAC field units, Central Evaluator (Central Evaluator unit shall be housed in a pre-fabricated Rack to be supplied along with the System) Reset unit, Relay unit, Event logger and diagnostic Terminal etc. Re...",SOUTH EASTERN RAILWAY,India,"11, Garden Reach Road, Kolkata-700043, West Bengal, Attn.:A.K.Agarwal, Controller Of Stores",Original Document : Document,2018-07-23 00:00:00,None,"Thursday, Jun 28, 2018",tenders are invited for supply supervision installation testing and commissioning of multi section digital axel counter for 30 detection point and 19 track section complete with all accessories cable required with each track device and card viz axle detectors track side electronics dac field units central evaluator central evaluator unit shall be housed in a pre fabricated rack to be supplied along with the system reset unit relay unit event logger and diagnostic terminal etc re...,"[tender, are, invit, for, suppli, supervis, instal, test, and, commiss, of, multi, section, digit, axel, counter, for, 30, detect, point, and, 19, track, section, complet, with, all, accessori, cabl, requir, with, each, track, devic, and, card, viz, axl, detector, track, side, electron, dac, field, unit, central, evalu, central, evalu, unit, shall, be, hous, in, a, pre, fabric, rack, to, be, suppli, along, with, the, system, reset, unit, relay, unit, event, logger, and, diagnost, termin, etc...",,
1,Domestic Competitive Bidding,2.India - Supply And Commissioning Of Gps Tracking Device.,"Tenders are invited for Supply and commissioning of GPS tracking device.,Type: Open,Eligibility Criteria : -",SOUTH EASTERN RAILWAY,India,"11, Garden Reach Road, Kolkata-700043, West Bengal, Attn.:A.K.Agarwal, Controller Of Stores",Original Document : Document,2018-07-24 00:00:00,None,"Thursday, Jun 28, 2018",tenders are invited for supply and commissioning of gps tracking device type open eligibility criteria,"[tender, are, invit, for, suppli, and, commiss, of, gp, track, devic, type, open, elig, criteria]",,
2,Domestic Competitive Bidding,3.India - Providing Of Stand_alone,"Tenders are invited for Providing of Stand_alone,Tender Type: Open tender,Form Of Contract: Percentage,Tender Category: Works,No. of Covers: 2,General Technical Evaluation Allowed: No,ItemWise Technical Evaluation Allowed: No,Payment Mode: Online,Is Multi Currency Allowed For BOQ: No,Is Multi Currency Allowed For Fee: No EMD value : INR 55828 Document Purchase Start date : 29 Jun 2018 Document Purchase End date : 14 Jul 2018 Opening date : 16 Jul 2018",PUBLIC HEALTH ENGINEERING DEPARTMENT,India,"Hooghly Division, State: west Bengal Attn: Executive Engineer",Original Document,2018-07-14 00:00:00,Rs. 2791408,"Thursday, Jun 28, 2018",tenders are invited for providing of stand_alone tender type open tender form of contract percentage tender category works no of covers 2 general technical evaluation allowed no itemwise technical evaluation allowed no payment mode online is multi currency allowed for boq no is multi currency allowed for fee no emd value inr 55828 document purchase start date 29 jun 2018 document purchase end date 14 jul 2018 opening date 16 jul 2018,"[tender, are, invit, for, provid, of, stand_alon, tender, type, open, tender, form, of, contract, percentag, tender, categori, work, no, of, cov

In [52]:
data1.dtypes

Title
Bidding Type           object
Project Title          object
Description            object
Organisation           object
Project Location       object
Client Address         object
Related Documents :    object
Deadline               object
Estimated Cost         object
Date                   object
DescriptionA           object
text_stem              object
BU                     object
BD                     object
dtype: object

In [53]:
# data1['Deadline'] = pd.to_datetime(data1['Deadline'],format='%d%m%Y' )
data1['Deadline'] = pd.to_datetime(data1['Deadline'])
data1.head()

Title,Bidding Type,Project Title,Description,Organisation,Project Location,Client Address,Related Documents :,Deadline,Estimated Cost,Date,DescriptionA,text_stem,BU,BD
0,Domestic Competitive Bidding,"1.India - Supply, Supervision, Installation, Testing And Commissioning Of Multi Section Digital Axel Counter For 30 Detection Point And 19 Track Section Complete With All Accessories, Cable Required With Each","Tenders are invited for Supply, Supervision, Installation, Testing and commissioning of Multi section Digital Axel counter for 30 Detection point and 19 Track section complete with all accessories, Cable required with each track device and card (Viz. Axle Detectors, Track side Electronics/DAC field units, Central Evaluator (Central Evaluator unit shall be housed in a pre-fabricated Rack to be supplied along with the System) Reset unit, Relay unit, Event logger and diagnostic Terminal etc. Re...",SOUTH EASTERN RAILWAY,India,"11, Garden Reach Road, Kolkata-700043, West Bengal, Attn.:A.K.Agarwal, Controller Of Stores",Original Document : Document,2018-07-23,None,"Thursday, Jun 28, 2018",tenders are invited for supply supervision installation testing and commissioning of multi section digital axel counter for 30 detection point and 19 track section complete with all accessories cable required with each track device and card viz axle detectors track side electronics dac field units central evaluator central evaluator unit shall be housed in a pre fabricated rack to be supplied along with the system reset unit relay unit event logger and diagnostic terminal etc re...,"[tender, are, invit, for, suppli, supervis, instal, test, and, commiss, of, multi, section, digit, axel, counter, for, 30, detect, point, and, 19, track, section, complet, with, all, accessori, cabl, requir, with, each, track, devic, and, card, viz, axl, detector, track, side, electron, dac, field, unit, central, evalu, central, evalu, unit, shall, be, hous, in, a, pre, fabric, rack, to, be, suppli, along, with, the, system, reset, unit, relay, unit, event, logger, and, diagnost, termin, etc...",,
1,Domestic Competitive Bidding,2.India - Supply And Commissioning Of Gps Tracking Device.,"Tenders are invited for Supply and commissioning of GPS tracking device.,Type: Open,Eligibility Criteria : -",SOUTH EASTERN RAILWAY,India,"11, Garden Reach Road, Kolkata-700043, West Bengal, Attn.:A.K.Agarwal, Controller Of Stores",Original Document : Document,2018-07-24,None,"Thursday, Jun 28, 2018",tenders are invited for supply and commissioning of gps tracking device type open eligibility criteria,"[tender, are, invit, for, suppli, and, commiss, of, gp, track, devic, type, open, elig, criteria]",,
2,Domestic Competitive Bidding,3.India - Providing Of Stand_alone,"Tenders are invited for Providing of Stand_alone,Tender Type: Open tender,Form Of Contract: Percentage,Tender Category: Works,No. of Covers: 2,General Technical Evaluation Allowed: No,ItemWise Technical Evaluation Allowed: No,Payment Mode: Online,Is Multi Currency Allowed For BOQ: No,Is Multi Currency Allowed For Fee: No EMD value : INR 55828 Document Purchase Start date : 29 Jun 2018 Document Purchase End date : 14 Jul 2018 Opening date : 16 Jul 2018",PUBLIC HEALTH ENGINEERING DEPARTMENT,India,"Hooghly Division, State: west Bengal Attn: Executive Engineer",Original Document,2018-07-14,Rs. 2791408,"Thursday, Jun 28, 2018",tenders are invited for providing of stand_alone tender type open tender form of contract percentage tender category works no of covers 2 general technical evaluation allowed no itemwise technical evaluation allowed no payment mode online is multi currency allowed for boq no is multi currency allowed for fee no emd value inr 55828 document purchase start date 29 jun 2018 document purchase end date 14 jul 2018 opening date 16 jul 2018,"[tender, are, invit, for, provid, of, stand_alon, tender, type, open, tender, form, of, contract, percentag, tender, categori, work, no, of, cover, 2, gener, technic, eval

In [54]:
# data1["TenderScrutinyDate"]=pd.to_datetime("2018-06-22" )
data1["Date"]=pd.to_datetime(data1.Date)
data1.head()

Title,Bidding Type,Project Title,Description,Organisation,Project Location,Client Address,Related Documents :,Deadline,Estimated Cost,Date,DescriptionA,text_stem,BU,BD
0,Domestic Competitive Bidding,"1.India - Supply, Supervision, Installation, Testing And Commissioning Of Multi Section Digital Axel Counter For 30 Detection Point And 19 Track Section Complete With All Accessories, Cable Required With Each","Tenders are invited for Supply, Supervision, Installation, Testing and commissioning of Multi section Digital Axel counter for 30 Detection point and 19 Track section complete with all accessories, Cable required with each track device and card (Viz. Axle Detectors, Track side Electronics/DAC field units, Central Evaluator (Central Evaluator unit shall be housed in a pre-fabricated Rack to be supplied along with the System) Reset unit, Relay unit, Event logger and diagnostic Terminal etc. Re...",SOUTH EASTERN RAILWAY,India,"11, Garden Reach Road, Kolkata-700043, West Bengal, Attn.:A.K.Agarwal, Controller Of Stores",Original Document : Document,2018-07-23,None,2018-06-28,tenders are invited for supply supervision installation testing and commissioning of multi section digital axel counter for 30 detection point and 19 track section complete with all accessories cable required with each track device and card viz axle detectors track side electronics dac field units central evaluator central evaluator unit shall be housed in a pre fabricated rack to be supplied along with the system reset unit relay unit event logger and diagnostic terminal etc re...,"[tender, are, invit, for, suppli, supervis, instal, test, and, commiss, of, multi, section, digit, axel, counter, for, 30, detect, point, and, 19, track, section, complet, with, all, accessori, cabl, requir, with, each, track, devic, and, card, viz, axl, detector, track, side, electron, dac, field, unit, central, evalu, central, evalu, unit, shall, be, hous, in, a, pre, fabric, rack, to, be, suppli, along, with, the, system, reset, unit, relay, unit, event, logger, and, diagnost, termin, etc...",,
1,Domestic Competitive Bidding,2.India - Supply And Commissioning Of Gps Tracking Device.,"Tenders are invited for Supply and commissioning of GPS tracking device.,Type: Open,Eligibility Criteria : -",SOUTH EASTERN RAILWAY,India,"11, Garden Reach Road, Kolkata-700043, West Bengal, Attn.:A.K.Agarwal, Controller Of Stores",Original Document : Document,2018-07-24,None,2018-06-28,tenders are invited for supply and commissioning of gps tracking device type open eligibility criteria,"[tender, are, invit, for, suppli, and, commiss, of, gp, track, devic, type, open, elig, criteria]",,
2,Domestic Competitive Bidding,3.India - Providing Of Stand_alone,"Tenders are invited for Providing of Stand_alone,Tender Type: Open tender,Form Of Contract: Percentage,Tender Category: Works,No. of Covers: 2,General Technical Evaluation Allowed: No,ItemWise Technical Evaluation Allowed: No,Payment Mode: Online,Is Multi Currency Allowed For BOQ: No,Is Multi Currency Allowed For Fee: No EMD value : INR 55828 Document Purchase Start date : 29 Jun 2018 Document Purchase End date : 14 Jul 2018 Opening date : 16 Jul 2018",PUBLIC HEALTH ENGINEERING DEPARTMENT,India,"Hooghly Division, State: west Bengal Attn: Executive Engineer",Original Document,2018-07-14,Rs. 2791408,2018-06-28,tenders are invited for providing of stand_alone tender type open tender form of contract percentage tender category works no of covers 2 general technical evaluation allowed no itemwise technical evaluation allowed no payment mode online is multi currency allowed for boq no is multi currency allowed for fee no emd value inr 55828 document purchase start date 29 jun 2018 document purchase end date 14 jul 2018 opening date 16 jul 2018,"[tender, are, invit, for, provid, of, stand_alon, tender, type, open, tender, form, of, contract, percentag, tender, categori, work, no, of, cover, 2, gener, technic, evalu, allow, no, itemwis, technic, evalu, all

In [55]:
data1["DaysLeft"] = data1['Deadline'].sub(data1["Date"], axis=0)
data1.head()

Title,Bidding Type,Project Title,Description,Organisation,Project Location,Client Address,Related Documents :,Deadline,Estimated Cost,Date,DescriptionA,text_stem,BU,BD,DaysLeft
0,Domestic Competitive Bidding,"1.India - Supply, Supervision, Installation, Testing And Commissioning Of Multi Section Digital Axel Counter For 30 Detection Point And 19 Track Section Complete With All Accessories, Cable Required With Each","Tenders are invited for Supply, Supervision, Installation, Testing and commissioning of Multi section Digital Axel counter for 30 Detection point and 19 Track section complete with all accessories, Cable required with each track device and card (Viz. Axle Detectors, Track side Electronics/DAC field units, Central Evaluator (Central Evaluator unit shall be housed in a pre-fabricated Rack to be supplied along with the System) Reset unit, Relay unit, Event logger and diagnostic Terminal etc. Re...",SOUTH EASTERN RAILWAY,India,"11, Garden Reach Road, Kolkata-700043, West Bengal, Attn.:A.K.Agarwal, Controller Of Stores",Original Document : Document,2018-07-23,None,2018-06-28,tenders are invited for supply supervision installation testing and commissioning of multi section digital axel counter for 30 detection point and 19 track section complete with all accessories cable required with each track device and card viz axle detectors track side electronics dac field units central evaluator central evaluator unit shall be housed in a pre fabricated rack to be supplied along with the system reset unit relay unit event logger and diagnostic terminal etc re...,"[tender, are, invit, for, suppli, supervis, instal, test, and, commiss, of, multi, section, digit, axel, counter, for, 30, detect, point, and, 19, track, section, complet, with, all, accessori, cabl, requir, with, each, track, devic, and, card, viz, axl, detector, track, side, electron, dac, field, unit, central, evalu, central, evalu, unit, shall, be, hous, in, a, pre, fabric, rack, to, be, suppli, along, with, the, system, reset, unit, relay, unit, event, logger, and, diagnost, termin, etc...",,,25 days
1,Domestic Competitive Bidding,2.India - Supply And Commissioning Of Gps Tracking Device.,"Tenders are invited for Supply and commissioning of GPS tracking device.,Type: Open,Eligibility Criteria : -",SOUTH EASTERN RAILWAY,India,"11, Garden Reach Road, Kolkata-700043, West Bengal, Attn.:A.K.Agarwal, Controller Of Stores",Original Document : Document,2018-07-24,None,2018-06-28,tenders are invited for supply and commissioning of gps tracking device type open eligibility criteria,"[tender, are, invit, for, suppli, and, commiss, of, gp, track, devic, type, open, elig, criteria]",,,26 days
2,Domestic Competitive Bidding,3.India - Providing Of Stand_alone,"Tenders are invited for Providing of Stand_alone,Tender Type: Open tender,Form Of Contract: Percentage,Tender Category: Works,No. of Covers: 2,General Technical Evaluation Allowed: No,ItemWise Technical Evaluation Allowed: No,Payment Mode: Online,Is Multi Currency Allowed For BOQ: No,Is Multi Currency Allowed For Fee: No EMD value : INR 55828 Document Purchase Start date : 29 Jun 2018 Document Purchase End date : 14 Jul 2018 Opening date : 16 Jul 2018",PUBLIC HEALTH ENGINEERING DEPARTMENT,India,"Hooghly Division, State: west Bengal Attn: Executive Engineer",Original Document,2018-07-14,Rs. 2791408,2018-06-28,tenders are invited for providing of stand_alone tender type open tender form of contract percentage tender category works no of covers 2 general technical evaluation allowed no itemwise technical evaluation allowed no payment mode online is multi currency allowed for boq no is multi currency allowed for fee no emd value inr 55828 document purchase start date 29 jun 2018 document purchase end date 14 jul 2018 opening date 16 jul 2018,"[tender, are, invit, for, provid, of, stand_alon, tender, type, open, tender, form, of, contract, percentag, tender, categori, work, no, of, cover, 2, gener, technic, evalu, allow, no, ite

In [56]:
columnsTitles=["Description",'Bidding Type',"Project Title","Organisation","Project Location","Client Address","Estimated Cost","BU","BD","Deadline","Date","DaysLeft",'Related Documents :']
data1=data1.reindex(columns=columnsTitles)

In [57]:
data1[data1.BU != " "]

Title,Description,Bidding Type,Project Title,Organisation,Project Location,Client Address,Estimated Cost,BU,BD,Deadline,Date,DaysLeft,Related Documents :
13,"Tenders are invited for Supervision of gbmp transport and business activities and information technology based management reporting,Capping value package rp: 1.178.880.000,00 Hps value package rp: 661.683.000,00",Not Specified Domestic / International Bidding ?,14.Indonesia - Supervision Of Gbmp Transport And Business Activities And Information Technology Based Management Reporting,KEMENTERIAN ENERGI DAN SUMBER DAYA MINERAL,Indonesia,project,"IDR, 661683000",BD2,Check,2018-07-23,2018-06-28,25 days,Original Document
15,"Prequalification for: (re-auction) Cost of consultancy services of relocation planning of puskesmas Development and house of medical service / paramedic of puskesmas batoh,Capping value package rp: 472.499.300,00 Hps value package rp: 462.550.000,00",Not Specified Domestic / International Bidding ?,16.Indonesia - (re-auction) Cost Of Consultancy Services Of Relocation Planning Of Puskesmas Development And House Of Medical Service / Paramedic Of Puskesmas Batoh,PEMERINTAH DAERAH KOTA BANDA ACEH,Indonesia,Kota Banda Aceh - Aceh Barat Daya (Kab.),"IDR, 462550000",Engg,Anand,2018-07-04,2018-06-28,6 days,Original Document
17,"Prequalification for: consultant services advanced supervisor of central wall road construction,Capping value package rp: 300.000.000,00 Hps value package rp: 254.925.000,00",Not Specified Domestic / International Bidding ?,18.Indonesia - Consultant Services Advanced Supervisor Of Central Wall Road Construction,PEMERINTAH DAERAH KOTA SAMARINDA,Indonesia,Samarinda - Samarinda (Kota),"IDR, 254925000",Engg,Anand,2018-07-04,2018-06-28,6 days,Original Document
21,Tender are Invited for for the Provision of Planning Services for the Preparation of A Master Plan for A Southern Access Road Project for the City for the Manager of the Roof Agreement in the City of Ashdod,Not Specified Domestic / International Bidding ?,22.Israel - Provision Of Planning Services For The Preparation Of A Master Plan For A Southern Access Road Project For The City For The Manager Of The Roof Agreement In The City Of Ashdod,THE MUNICIPALITY OF ASHDOD,Israel,"City Hall, 3rd Floor Room 318.,Tel: +972-8545257/8545258/732255306/6523614,Fax: +972-88545260/6523542,Email_id: contact@ashdod.muni.il/yonatanl@ashdod.muni.il/contact_ashdod@ashdod.muni.il",None,TP,Ashish,2018-07-26,2018-06-28,28 days,Original Document
24,"Tender notice Procurement Subject And Quantity Of Air traffic Management Processing System (Team) Performance Improvement, Production And Adjustment 1 Set Of Equipment Set",Not Specified Domestic / International Bidding ?,25.Japan - Air Traffic Management Processing System,"MINISTRY OF LAND, INFRASTRUCTURE AND TRANSPORT",Japan,"Yubinbango100-8918, Chiyoda-ku, Tokyo Kasumigaseki 2-1-3",None,ITS,Gurpreet,2018-08-02,2018-06-28,35 days,Original Document
26,"Procurement Notices: Lease for next generation firewall, etc. In bureau of transportation 1 set",Not Specified Domestic / International Bidding ?,"27.Japan - Lease For Next Generation Firewall, Etc. In Bureau Of Transportation 1 Set",TOKYO-TO,Japan,112–0001 Tokyo Bunkyo–Ku Hakusan 4–Chome 3–2.,None,BD2,Check,2018-08-07,2018-06-28,40 days,Original Document
31,"Prefectural road lobby point pollution reduction facility master plan establishment service of the sphere of influence,Estimated cost: 537,721,818 won,Bidding deadline: 2018/07/18 11:00,Wicket (bid) Date: 2018/07/18 17:00",International Competitive Bidding,32.Korea Republic of - Prefectural Road Lobby Point Pollution Reduction Facility Master Plan Establishment Service Of The Sphere Of Influence,GYEONGGI PROVINCIAL GOVERNMENT OFFICE,Korea Republic of,"Building 3, Government Complex-Daejeon, 189 Cheoungsa-ro, Seo-gu, Daejeon, Korea.","KRW, 537721818",TP,Ashish,2018-07-18,2018-06-28,20 days,Original Document
39,"Prequalification are Invited for Supply of Master P

In [58]:
# from nltk import PorterStemmer
# PorterStemmer().stem('child')

In [59]:
data2 = data1[data1.BU != " "]
data2

Title,Description,Bidding Type,Project Title,Organisation,Project Location,Client Address,Estimated Cost,BU,BD,Deadline,Date,DaysLeft,Related Documents :
13,"Tenders are invited for Supervision of gbmp transport and business activities and information technology based management reporting,Capping value package rp: 1.178.880.000,00 Hps value package rp: 661.683.000,00",Not Specified Domestic / International Bidding ?,14.Indonesia - Supervision Of Gbmp Transport And Business Activities And Information Technology Based Management Reporting,KEMENTERIAN ENERGI DAN SUMBER DAYA MINERAL,Indonesia,project,"IDR, 661683000",BD2,Check,2018-07-23,2018-06-28,25 days,Original Document
15,"Prequalification for: (re-auction) Cost of consultancy services of relocation planning of puskesmas Development and house of medical service / paramedic of puskesmas batoh,Capping value package rp: 472.499.300,00 Hps value package rp: 462.550.000,00",Not Specified Domestic / International Bidding ?,16.Indonesia - (re-auction) Cost Of Consultancy Services Of Relocation Planning Of Puskesmas Development And House Of Medical Service / Paramedic Of Puskesmas Batoh,PEMERINTAH DAERAH KOTA BANDA ACEH,Indonesia,Kota Banda Aceh - Aceh Barat Daya (Kab.),"IDR, 462550000",Engg,Anand,2018-07-04,2018-06-28,6 days,Original Document
17,"Prequalification for: consultant services advanced supervisor of central wall road construction,Capping value package rp: 300.000.000,00 Hps value package rp: 254.925.000,00",Not Specified Domestic / International Bidding ?,18.Indonesia - Consultant Services Advanced Supervisor Of Central Wall Road Construction,PEMERINTAH DAERAH KOTA SAMARINDA,Indonesia,Samarinda - Samarinda (Kota),"IDR, 254925000",Engg,Anand,2018-07-04,2018-06-28,6 days,Original Document
21,Tender are Invited for for the Provision of Planning Services for the Preparation of A Master Plan for A Southern Access Road Project for the City for the Manager of the Roof Agreement in the City of Ashdod,Not Specified Domestic / International Bidding ?,22.Israel - Provision Of Planning Services For The Preparation Of A Master Plan For A Southern Access Road Project For The City For The Manager Of The Roof Agreement In The City Of Ashdod,THE MUNICIPALITY OF ASHDOD,Israel,"City Hall, 3rd Floor Room 318.,Tel: +972-8545257/8545258/732255306/6523614,Fax: +972-88545260/6523542,Email_id: contact@ashdod.muni.il/yonatanl@ashdod.muni.il/contact_ashdod@ashdod.muni.il",None,TP,Ashish,2018-07-26,2018-06-28,28 days,Original Document
24,"Tender notice Procurement Subject And Quantity Of Air traffic Management Processing System (Team) Performance Improvement, Production And Adjustment 1 Set Of Equipment Set",Not Specified Domestic / International Bidding ?,25.Japan - Air Traffic Management Processing System,"MINISTRY OF LAND, INFRASTRUCTURE AND TRANSPORT",Japan,"Yubinbango100-8918, Chiyoda-ku, Tokyo Kasumigaseki 2-1-3",None,ITS,Gurpreet,2018-08-02,2018-06-28,35 days,Original Document
26,"Procurement Notices: Lease for next generation firewall, etc. In bureau of transportation 1 set",Not Specified Domestic / International Bidding ?,"27.Japan - Lease For Next Generation Firewall, Etc. In Bureau Of Transportation 1 Set",TOKYO-TO,Japan,112–0001 Tokyo Bunkyo–Ku Hakusan 4–Chome 3–2.,None,BD2,Check,2018-08-07,2018-06-28,40 days,Original Document
31,"Prefectural road lobby point pollution reduction facility master plan establishment service of the sphere of influence,Estimated cost: 537,721,818 won,Bidding deadline: 2018/07/18 11:00,Wicket (bid) Date: 2018/07/18 17:00",International Competitive Bidding,32.Korea Republic of - Prefectural Road Lobby Point Pollution Reduction Facility Master Plan Establishment Service Of The Sphere Of Influence,GYEONGGI PROVINCIAL GOVERNMENT OFFICE,Korea Republic of,"Building 3, Government Complex-Daejeon, 189 Cheoungsa-ro, Seo-gu, Daejeon, Korea.","KRW, 537721818",TP,Ashish,2018-07-18,2018-06-28,20 days,Original Document
39,"Prequalification are Invited for Supply of Master P

In [60]:
data2.sort_values(["Project Location","BU"], inplace=True)

In [61]:
data2

Title,Description,Bidding Type,Project Title,Organisation,Project Location,Client Address,Estimated Cost,BU,BD,Deadline,Date,DaysLeft,Related Documents :
300,"Expressions of interest for Francophone Short Term Consultant to support Advisory activities in MENA and Sub-Saharan Africa,Description: Fig Advisory Program Is Seeking to Recruit a Short-term Consultant (Stc) to Provide Support to the Banking & Risk Management and Digital Financial Services Practice Groups (Pg)in Mena and Sub-saharan Africa. Specific Role and Responsibilities Include: - Engagement Management and Development Impact: Support Program Managers, Operations Officer and Senior Spe...",Not Specified Domestic / International Bidding ?,"1.Algeria,Burkina Faso,Cote Dlvoire,Morocco,Senegal,Tunisia - Francophone Short Term Consultant To Support Advisory Activities",WORLD BANK (WB),"Algeria,Burkina Faso,Cote Dlvoire,Morocco,Senegal,Tunisia","7, Rue Larbi Ben Abdellah, Souissi-Rabat,,Tel: +212-537-544-200",None,BD2,Check,2018-07-15,2018-06-28,17 days,Original Document
364,"Expression of Interest for Consulting services for design, implementation and supervisory support of solar photovoltaic pumping for agricultural irrigation (firm),Description: Objective and Purpose of the Assignment: The Consultant for Design, Implementation and Supervisory Support of Solar Photovoltaic Pumping for Agricultural Irrigation Project will prepare the design and assist BREB in implementing and ensuring engineering oversight of the SPV pumping system installation, commissioning an...",International Competitive Bidding,"2.Bangladesh - Consulting Services For Design, Implementation And Supervisory Support Of Solar Photovoltaic Pumping",BANGLADESH RURAL ELECTRIFICATION BOARD,Bangladesh,"Directorate of Procurement Head Office Bldg.-7th Flr, Zoar Sahara, Khilkhet, Dhaka-1229, Bangladesh,Tel: +880-1749243881","BDT, 708568",BD2,Check,2018-07-27,2018-06-28,29 days,Original Document
260,"Request for quotations : Development of town planning documentation draft detailed planning of the area in the area of bogdanovicha street, bounded by the floodplain of the sluch river and the microdistrict of the individual housing estate novodvorets in the town of slutsk, minsk region.,1. Development of town planning documentation draft detailed planning of the area in the area of bogdanovicha street, bounded by floodplain of the river sluch and microdistrict of the individual housing esta...",Not Specified Domestic / International Bidding ?,"4.Belarus - Development Of Town Planning Documentation Draft Detailed Planning Of The Area In The Area Of Bogdanovicha Street, Bounded By The Floodplain Of The Sluch River And The Microdistrict Of The Individual",SLUTSK REGIONAL EXECUTIVE COMMITTEE,Belarus,"Belarus, 223610, Slutsk, Lenin St., 189,Tel: +375-95 63177","BYR, 52000",TP,Ashish,2018-07-12,2018-06-28,14 days,Original Document : Bid Document
270,"Tenders are invited for Analysis of the compliance of the existing systems and processes in BDZ-Passenger transport EOOD with the requirements of Regulation (EC) 2016/679 of the European Parliament and of the Council on the protection of individuals with regard to the processing of personal data and on the ""f r è è"" movement of such data (GDPR) and defining measures to bring operational processes and information systems in line with the Regulation.,Total estimated value of the order,70000 BG...",Not Specified Domestic / International Bidding ?,14.Bulgaria - Analysis Of The Compliance Of The Existing Systems And Processes In Bdz-passenger Transport Eood With The Requirements Of Regulation (ec) 2016/679 Of The European Parliament And Of The Council On The,BDZ - PASSENGER TRANSPORT EOOD,Bulgaria,"Bulgaria 1080, Sofia,Tel: +359-0889 077505,Fax: +359-02 9800494,Email_id: mailto:krivanova@bdz.bg","BGL, 70000",BD2,Check,2018-07-05,2018-06-28,7 days,Original Document
269,"Contract notice: preparation of a technical investment project for a stadium and sports facility in regulate

In [62]:
# nreg=[]
nreg=['Australia','Taiwan Province of China','Brazil','Canada','Germany','Denmark','France','United States','Italy','Finland','Czech Republic','China','Belgium','Belarus','New Zealand','Russian Federation','Japan']

In [63]:
# final=data2.loc[data2["Project Location"] != ['Australia','Canada','Germany','France','United States','Italy','Finland','Czech Republic','China','Belgium','Belarus']]
# final.drop(['text_stem','Project Title'], axis=1, inplace=True)
final=data2[~data2["Project Location"].isin(nreg)]

In [64]:
final = final.reset_index(drop=True)
final.head()

Title,Description,Bidding Type,Project Title,Organisation,Project Location,Client Address,Estimated Cost,BU,BD,Deadline,Date,DaysLeft,Related Documents :
0,"Expressions of interest for Francophone Short Term Consultant to support Advisory activities in MENA and Sub-Saharan Africa,Description: Fig Advisory Program Is Seeking to Recruit a Short-term Consultant (Stc) to Provide Support to the Banking & Risk Management and Digital Financial Services Practice Groups (Pg)in Mena and Sub-saharan Africa. Specific Role and Responsibilities Include: - Engagement Management and Development Impact: Support Program Managers, Operations Officer and Senior Spe...",Not Specified Domestic / International Bidding ?,"1.Algeria,Burkina Faso,Cote Dlvoire,Morocco,Senegal,Tunisia - Francophone Short Term Consultant To Support Advisory Activities",WORLD BANK (WB),"Algeria,Burkina Faso,Cote Dlvoire,Morocco,Senegal,Tunisia","7, Rue Larbi Ben Abdellah, Souissi-Rabat,,Tel: +212-537-544-200",None,BD2,Check,2018-07-15,2018-06-28,17 days,Original Document
1,"Expression of Interest for Consulting services for design, implementation and supervisory support of solar photovoltaic pumping for agricultural irrigation (firm),Description: Objective and Purpose of the Assignment: The Consultant for Design, Implementation and Supervisory Support of Solar Photovoltaic Pumping for Agricultural Irrigation Project will prepare the design and assist BREB in implementing and ensuring engineering oversight of the SPV pumping system installation, commissioning an...",International Competitive Bidding,"2.Bangladesh - Consulting Services For Design, Implementation And Supervisory Support Of Solar Photovoltaic Pumping",BANGLADESH RURAL ELECTRIFICATION BOARD,Bangladesh,"Directorate of Procurement Head Office Bldg.-7th Flr, Zoar Sahara, Khilkhet, Dhaka-1229, Bangladesh,Tel: +880-1749243881","BDT, 708568",BD2,Check,2018-07-27,2018-06-28,29 days,Original Document
2,"Tenders are invited for Analysis of the compliance of the existing systems and processes in BDZ-Passenger transport EOOD with the requirements of Regulation (EC) 2016/679 of the European Parliament and of the Council on the protection of individuals with regard to the processing of personal data and on the ""f r è è"" movement of such data (GDPR) and defining measures to bring operational processes and information systems in line with the Regulation.,Total estimated value of the order,70000 BG...",Not Specified Domestic / International Bidding ?,14.Bulgaria - Analysis Of The Compliance Of The Existing Systems And Processes In Bdz-passenger Transport Eood With The Requirements Of Regulation (ec) 2016/679 Of The European Parliament And Of The Council On The,BDZ - PASSENGER TRANSPORT EOOD,Bulgaria,"Bulgaria 1080, Sofia,Tel: +359-0889 077505,Fax: +359-02 9800494,Email_id: mailto:krivanova@bdz.bg","BGL, 70000",BD2,Check,2018-07-05,2018-06-28,7 days,Original Document
3,"Contract notice: preparation of a technical investment project for a stadium and sports facility in regulated area iv-66,71, district 4a according to the plan of the south west wp, pi with identifier 07079.663.697 by the ccrc of bourgas and leading infrastructure,The project includes the preparation of a technical investment project for a stadium and a sports facility in reg. iv-66,71, district 4a according to the plan of the southwest lp, id with identifier 07079.663.697 under the bcrc of b...",International Competitive Bidding,"13.Bulgaria - Preparation Of A Technical Investment Project For A Stadium And Sports Facility In Regulated Area Iv-66,71, District 4a According To The Plan Of The South West Wp, Pi With Identifier 07079.663.697 By",OBSHTINA BURGAS (000056814),Bulgaria,"000056814 Ul. „Aleksandrovska No. 26 Burgas 8000 Contact Person: Aneliya Angelova,Tel: +359 56907288,Fax: +359 56860912,Email_id: a.angelova@burgas.bg",None,Rail,Anand,2018-08-01,2018-06-28,34 days,Original Document
4,"Expression of Interest for Trade and transport facilitation speciali

In [65]:
data4 = data1[data1.BU == " "]
data4.sort_values(["Project Location","BU"], inplace=True)
final2=data4
# final2.drop(['text_stem','Project Title'], axis=1, inplace=True)
final2=data4[~data4["Project Location"].isin(nreg)]
final2 = final2.reset_index(drop=True)


In [66]:
dataA = final[final.BD == "Ashish"].reset_index(drop=True)
dataK = final[final.BD == "Anand"].reset_index(drop=True)
dataS = final[final.BD == "Saumya"].reset_index(drop=True)
dataG = final[final.BD == "Gurpreet"].reset_index(drop=True)
dataP = final[final.BD == "Sudeep"].reset_index(drop=True)
dataB=  final[final.BD == "All"].reset_index(drop=True)
dataC=  final[final.BD == "Check"].reset_index(drop=True)

In [67]:
from pandas import ExcelWriter
writer = ExcelWriter('OppurtinityList.xlsx')
final.to_excel(writer,'Oppurtunity')
final2.to_excel(writer,'Not_Oppurtinity')
dataA.to_excel(writer,'Ashish')
dataK.to_excel(writer,'Anand')
dataS.to_excel(writer,'Saumya')
dataG.to_excel(writer,'Gurpreet')
dataP.to_excel(writer,'Sudeep')
dataB.to_excel(writer,'All')
dataC.to_excel(writer,'Check')
writer.save()

In [68]:
# from pandas import ExcelWriter
# writer = ExcelWriter('OppurtinityList.xlsx')
# final.to_excel(writer,'Oppurtunity')
# final2.to_excel(writer,'Not_Oppurtinity')
# writer.save()